In [15]:
function A(x::Int64, y::String)
    return (x, y)
end

A (generic function with 1 method)

In [16]:
Base.return_types(A, (Int64, String))[1]

Tuple{Int64, String}

In [13]:
A((1,"hi")...)

1

In [18]:
Channel{Base.return_types(A, (Int64, String))[1]}(32)

Channel{Tuple{Int64, String}}(32) (empty)

In [1]:
import Pkg
Pkg.activate(".")

  Activating project at `~/Developer/DistributedStreams`


In [2]:
using Distributed

In [3]:
addprocs(2)

2-element Vector{Int64}:
 2
 3

In [4]:
@everywhere using Distributed

@everywhere function remote_worker(fn, entries, results)
    while true
        # take data from remote channel asynchronously
        t = @async take!(entries)
        # process data and return result in output channel -- and sync loop
        @sync put!(results, fn(fetch(t)))
    end
end

In [5]:
ch_in = RemoteChannel(()->Channel{Int64}(32), 1)
ch_out = RemoteChannel(()->Channel{Int64}(32), 1)

RemoteChannel{Channel{Int64}}(1, 1, 13)

In [6]:
@async while true
    t = take!(ch_out)
    println("Taken: $(t)")
end

Task (runnable) @0x00007ca37a0312d0

In [7]:
@everywhere function test_fn(i)
    println("hi there, I'm running on pid=$(myid())")
    return i+1
end

In [8]:
status_2 = remotecall(remote_worker, 2,
    test_fn, ch_in, ch_out
)

status_3 = remotecall(remote_worker, 3,
    test_fn, ch_in, ch_out
)

status_2_t = @async fetch(status_2)
status_3_t = @async fetch(status_3)

Task (runnable) @0x00007ca37ad54c90

In [9]:
put!(ch_in, 1)

RemoteChannel{Channel{Int64}}(1, 1, 12)

In [10]:
put!(ch_in, 2)

RemoteChannel{Channel{Int64}}(1, 1, 12)

      From worker 3:	hi there, I'm running on pid=3
      From worker 2:	hi there, I'm running on pid=2
Taken: 2
Taken: 3


In [11]:
rmprocs(3)

Task (done) @0x00007ca378b33d00

In [12]:
put!(ch_in, 3)

RemoteChannel{Channel{Int64}}(1, 1, 12)

      From worker 2:	hi there, I'm running on pid=2
Taken: 4


In [13]:
put!(ch_in, 4)

┌ Error: Fatal error on process 1
│   exception =
│    attempt to send to unknown socket
│    Stacktrace:
│     [1] error(s::String)
│       @ Base ./error.jl:35
│     [2] send_msg_unknown(s::Sockets.TCPSocket, header::Distributed.MsgHeader, msg::Distributed.ResultMsg)
│       @ Distributed ~/local/juliaup/juliaup/julia-1.10.2+0.x64.linux.gnu/share/julia/stdlib/v1.10/Distributed/src/messages.jl:99
│     [3] send_msg_now(s::Sockets.TCPSocket, header::Distributed.MsgHeader, msg::Distributed.ResultMsg)
│       @ Distributed ~/local/juliaup/juliaup/julia-1.10.2+0.x64.linux.gnu/share/julia/stdlib/v1.10/Distributed/src/messages.jl:115
│     [4] deliver_result(sock::Sockets.TCPSocket, msg::Symbol, oid::Distributed.RRID, value::Int64)
│       @ Distributed ~/local/juliaup/juliaup/julia-1.10.2+0.x64.linux.gnu/share/julia/stdlib/v1.10/Distributed/src/process_messages.jl:102
│     [5] (::Distributed.var"#109#111"{Distributed.CallMsg{:call_fetch}, Distributed.MsgHeader, Sockets.TCPSocket})()
│    

RemoteChannel{Channel{Int64}}(1, 1, 12)

In [14]:
put!(ch_in, 5)

RemoteChannel{Channel{Int64}}(1, 1, 12)

      From worker 2:	hi there, I'm running on pid=2
Taken: 6


In [15]:
put!(ch_in, 6)

RemoteChannel{Channel{Int64}}(1, 1, 12)

      From worker 2:	hi there, I'm running on pid=2
Taken: 7


In [16]:
put!(ch_in, 7)

RemoteChannel{Channel{Int64}}(1, 1, 12)

      From worker 2:	hi there, I'm running on pid=2
Taken: 8
